In [6]:
import re 
from zhon.hanzi import punctuation #中文标点符号处理
import numpy as np 
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB #多项式朴素贝叶斯分类器
from sklearn.metrics.pairwise import cosine_similarity
import editdistance
import pickle # 用于python特有的类型和python的数据类型间进行转换
from tqdm import tqdm # 进度条库，可以在 Python 长循环中添加一个进度提示信息用法：tqdm(iterator)

In [15]:
# 停用词
with open('E:/data/datasource/chinese_stopwords.txt','r',encoding='utf-8') as file:
    stopwords=[i[:-1] for i in file.readlines()]

In [7]:
news = pd.read_csv('E:/data/sqlResult_1558435.csv',encoding='gb18030')

In [8]:
news.shape

(89611, 7)

In [10]:
news.head(5)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [11]:
news[news.content.isna()].head(5)

,id,author,source,content,feature,title,url
100,89517,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",天和防务股东未来6个月内计划减持不超过480万股公司股份,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
103,89514,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",晶盛机电调整限制性股票回购价格,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
997,88620,NaN,央广网,NaN,"{""type"":""时事要闻"",""site"":""参考消息"",""commentNum"":""0"",...",[主播不在家]第二季：主播陈亮体验垃圾清运,http://www.cankaoxiaoxi.com/china/20170623/214...
1273,88344,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",LKK洛可可：想象力经济时代或已到来,http://www.cankaoxiaoxi.com/science/20170610/2...
1282,88335,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台,http://www.cankaoxiaoxi.com/science/20170610/2...


In [12]:
news=news.dropna(subset=['content'])

In [16]:
# 分词
def split_text(text): 
    return ' '.join([w for w in list(jieba.cut(re.sub('\s|[%s]' % (punctuation),'',text))) if w not in stopwords])

In [17]:
split_text(news.iloc[1].content)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\heyangm\AppData\Local\Temp\jieba.cache
Loading model cost 0.926 seconds.
Prefix dict has been built succesfully.


'骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 资料 显示 骁龙 835 一款 集成 CPUGPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台'

## prepare data 

In [19]:
corpus=list(map(split_text,[str(i) for i in news.content]))

In [18]:
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()
clf = MultinomialNB()

In [20]:
countvector = countvectorizer.fit_transform(corpus)

In [21]:
countvector.shape

(87054, 883)

In [22]:
tfidf = tfidftransformer.fit_transform(countvector)

In [23]:
tfidf.shape

(87054, 883)

In [24]:
# text label
label=list(map(lambda source: 1 if '新华' in str(source) else 0,news.source))

In [25]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3, random_state=42)

In [26]:
clf.fit(X=X_train,y=y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
scores=cross_validate(clf,X_train,y_train,scoring=('precision','recall','accuracy','f1'),cv=3,return_train_score=True)

In [28]:
scores

{'fit_time': array([0.35239434, 0.3581171 , 0.33487105]),
 'score_time': array([0.2133708 , 0.21389031, 0.20307851]),
 'test_precision': array([0.9599404 , 0.96233543, 0.96181975]),
 'train_precision': array([0.96242476, 0.96172716, 0.96269257]),
 'test_recall': array([0.91072205, 0.91409308, 0.90811222]),
 'train_recall': array([0.91286973, 0.91129295, 0.91055894]),
 'test_accuracy': array([0.88475361, 0.88981883, 0.88415715]),
 'train_accuracy': array([0.88883419, 0.88684308, 0.88706462]),
 'test_f1': array([0.93468374, 0.93759411, 0.9341947 ]),
 'train_f1': array([0.93699249, 0.93583104, 0.9359003 ])}

In [29]:
y_predict = clf.predict(X_test)

In [30]:
def show_test_reslt(y_true,y_pred):
    print('accuracy:',accuracy_score(y_true,y_pred))
    print('precison:',precision_score(y_true,y_pred))
    print('recall:',recall_score(y_true,y_pred))
    print('f1_score:',f1_score(y_true,y_pred))

In [31]:
show_test_reslt(y_test,y_predict)

accuracy: 0.8904162040050542
precison: 0.9624150339864055
recall: 0.9148612694792855
f1_score: 0.9380358534684333


## Use the model to detect copy news

In [33]:
prediction = clf.predict(tfidf.toarray())

labels = np.array(label)

compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})

copy_news_index=compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels'] == 0)].index

xinhuashe_news_index=compare_news_index[(compare_news_index['labels'] == 1)].index

In [34]:
len(copy_news_index)

2812

### In sklearn, in order to use cosine similarity in kmeans, we should scale the data to unit norm then use the kmeans class

In [35]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from collections import defaultdict # 定义默认字典

In [36]:
normalizer = Normalizer()

In [37]:
scaled_array = normalizer.fit_transform(tfidf.toarray())

In [38]:
kmeans = KMeans(n_clusters=25,random_state=42,n_jobs=-1)
k_labels = kmeans.fit_predict(scaled_array)

In [39]:
id_class = {index:class_ for index,class_ in enumerate(k_labels)}

In [40]:
class_id = defaultdict(set)
for index,class_ in id_class.items():
    if index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)

In [41]:
 def print_element(ids, n=10):
        count = 0
        for i in ids:
            if count > 10: break
            count+=1
            print(news.iloc[i].content, '\n')

In [48]:
def find_similar_text(cpindex, top=10):
    dist_dict = {i: cosine_similarity(tfidf[cpindex], tfidf[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(),key=lambda x:x[1][0],reverse=True)[:top]

In [50]:
CopyText = find_similar_text(3352)

In [51]:
CopyText

[(3134, array([[0.96849349]])),
 (63511, array([[0.94619604]])),
 (29441, array([[0.94281928]])),
 (3218, array([[0.87620818]])),
 (980, array([[0.87535143]])),
 (29615, array([[0.86922775]])),
 (29888, array([[0.86194742]])),
 (64046, array([[0.85277668]])),
 (29777, array([[0.84882241]])),
 (64758, array([[0.73406445]]))]

In [54]:
print('怀疑抄袭:\n')

print(news.iloc[3352].content)

print('相似原文:\n')

print(news.iloc[3134].content)

print('editdistince:',editdistance.eval(corpus[3352],corpus[3134]))

怀疑抄袭:

　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

相似原文:

　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。
　　“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建

## sentences similarity detection

In [55]:
def find_similar_sentence(candidate,raw):
    similist = []
    cl = candidate.strip().split('。')
    ra = raw.strip().split('。')
    for c in cl:
        for r in ra:
            similist.append([c,r,editdistance.eval(c,r)])
    sort=sorted(similist,key=lambda x:x[2])
    for c,r,ed in sort:
        if c!='' and r!='':
            print('怀疑抄袭句:{0}\n相似原句:{1}\neditdistance:{2}\n'.format(c,r,ed))

In [56]:
find_similar_sentence(news.iloc[3352].content,news.iloc[3134].content)

怀疑抄袭句:从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内
相似原句:从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内
editdistance:0

怀疑抄袭句:其中，一二线城市同比涨幅回落尤其明显
相似原句:其中，一二线城市同比涨幅回落尤其明显
editdistance:0

怀疑抄袭句:据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点
相似原句:据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点
editdistance:0

怀疑抄袭句:二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个
相似原句:二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个
editdistance:0

怀疑抄袭句:从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间
相似原句:”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间
editdistance:19

怀疑抄袭句:对此，国家统计局城市司高级统计师刘建伟进行了解读
相似原句:其中，一二线城市同比涨幅回落尤其明显
editdistance:21

怀疑抄袭句:从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间
相似原句:从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内
editdistance:25

怀疑抄袭句:二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个
相似原句:统计显示，5月份，70个大中城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个
editdistance:26

怀